In [ ]:
import confirm.outlaw.nb_util as nb_util
nb_util.setup_nb()
import xxhash

id = [0, 1]

In [ ]:
import confirm.cloud.clickhouse as ch
import pandas as pd
import numpy as np
# ch_db = ch.Clickhouse.connect()

In [ ]:
df = pd.read_parquet('research/cloud_explore/clickhouse/dbtest.parquet')

In [ ]:
import string
import random
# ids = [
#     ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(80))
#     for i in range(100000)
# ]
A, Z = np.array(["A","Z"]).view("int32") 

NO_CODES = df.shape[0]
LEN = 80

ids = np.random.randint(low=A,high=Z,size=NO_CODES*LEN,dtype="int32").view(f"U{LEN}")

In [ ]:
# ids_hashed = [xxhash.xxh128_digest(ids[i]) for i in range(ids.shape[0])]

In [ ]:
ch_db = ch.Clickhouse.connect(host='localhost', port=8123, password='')

In [ ]:
df['id'] = df.index
# df['id'] = ids
# df['id'] = df['id'].astype('|S80')
# df['id'] = ids_hashed
# df['id'] = df['id'].astype("|S16")
df['eligible'] = True
df['active'] = True
df['lineage'] = ids
df['lineage'] = df['id'].astype('|S80')
df['orderer'] = df['orig_lam']

In [ ]:
df.memory_usage().sum() / 1e9

7.646865994

In [ ]:
%%time
# ch_db.client.command('drop table tiles')
# ch_db.client.command('drop table tiles_inactive')
# ch_db.client.command('drop table work')
ch_db.init_tiles(df[:5000000])

Int64
CPU times: user 974 ms, sys: 696 ms, total: 1.67 s
Wall time: 3.74 s


In [ ]:
%%time
ch_db.write(df[5000000:10000000])
ch_db.write(df[10000000:15000000])
ch_db.write(df[15000000:20000000])

CPU times: user 2.72 s, sys: 2 s, total: 4.71 s
Wall time: 9.21 s


In [ ]:
%%time
ch_db.write(df[5000000:10000000])
ch_db.write(df[10000000:15000000])
ch_db.write(df[15000000:20000000])

CPU times: user 2.65 s, sys: 1.93 s, total: 4.59 s
Wall time: 9.12 s


In [ ]:
import time
runtimes = []
for i in range(5):
    start = time.time()
    ch_db.next(10000, 'orderer')
    print(i, time.time() - start)
    runtimes.append(time.time() - start)

0 0.10164499282836914
1 0.08141803741455078
2 0.08311700820922852
3 0.07768011093139648
4 0.08029007911682129


In [ ]:
import confirm.imprint as ip

In [ ]:
%%time
ip.grid.gen_short_uuids(10000)

CPU times: user 53 µs, sys: 3 µs, total: 56 µs
Wall time: 57 µs


array([10257294789557878784, 10257294789557878785, 10257294789557878786,
       ..., 10257294789557888781, 10257294789557888782,
       10257294789557888783], dtype=uint64)

In [ ]:
%%time
for i in range(10000):
    xxhash.xxh64_intdigest(str(id))

CPU times: user 2.66 ms, sys: 2 µs, total: 2.66 ms
Wall time: 2.66 ms
